파아토치는 tensorflow의 케라스 모듈과는 다르게 많이 수동적인 방법을 사용합니다.

특정 레이어를 쌓아서 모델을 설계하는 keras와는 다르게 파이토치는 직접 가중치 선언부터 연산까지 직접 설계해야합니다.

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1개의 입력 이미지, 6개의 필터링 이미지, 3x3 크기의 필터
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)

        # 단순 가중치 설정
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # tensorflow의 predict 와 동일기능
    def forward(self, x):

        # 맥스 풀링
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # ㄴ conv -> relu -> 2*2 사이즈 맥스 풀링
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # ㄴ 정사각형이라면 단일 정수 입력 가능
        x = x.view(-1, self.flatten(x))
        # ㄴ Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def flatten(self, x):
        size = x.size()[1:]
        result = 1
        for s in size:
            result *= s
        return result

In [46]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [47]:
param = list(net.parameters())
print(len(param))
for i in param:
    print(i.size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [48]:
input_img = torch.randn(1, 1, 32, 32)
out = net(input_img)
out

tensor([[ 0.0232,  0.0014,  0.0273, -0.0040, -0.0505, -0.0776, -0.1918,  0.0691,
          0.1226,  0.0217]], grad_fn=<AddmmBackward>)

In [49]:
net.zero_grad()
out.backward(torch.randn(1, 10))

Loss Function (손실 함수)

손실함수는 (예측 값, 실제 값)을 매개변수로 가지며 예측값이 실제값과 얼마나 많이 차이가 나는지 계산합니다.

손실함수에는 여러종류가 있지만 (nn 패키지에 들어 있습니다.) 가장 간단한 손실함수는 `nn.MSELoss`입니다.
수식은 다음과 같습니다.

$ y` = 예측값, y = 실제 값, n = 배열의 길이 $

$ L(y`_i, y_i) = {1 \over n} \Sigma^n_{i=1} (y_i - y`_i)^2 $

In [50]:
output = net(input_img)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

tensor(0.5303, grad_fn=<MseLossBackward>)

In [51]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Liner
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop (오차 역전파)

오차역전파는 `loss.backward()`로 얻어진 오차를 신경망에 감산합니다.



In [52]:
net.zero_grad() # 이전의 연산으로 생겨난 오차를 전부 제거합니다.

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0071, -0.0016, -0.0026,  0.0079, -0.0061, -0.0084])


그럼 이제 신경망을 학습시켜 봅시다.

신경망 학습은 대충 아래와 같은 방식으로 작동합니다.

In [53]:
learning_rate = 0.01 # 학습률입니다. 이 값이 작을수록 학습이 더디지만 더 정확하게 됩니다.
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

하지만 이 방법으로 가중치를 감산하는 것은 학습이 매우 더덥니다.

정말 다행히도 파이토치는 신경망 학습을 위한 여러가지 모듈을 지원합니다. 이를 옵티마이저(optimizer)이라고 합니다.

In [54]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input_img)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update